# Dynamic Programming

## Basics of Reinforcement Learning
Reinforcement learning is about taking suitable action to maximize reward in a particular situation. It is made up of three parts.\
**Agent** that interacts with the environment via actions.\
**Action space** is the set of all possible actions in an environment.Discrete spaces are finite whereas continuous states are infinite.\
**Environment** contains information about all possible actions that an agent can take, the states that agent can achieve following those actions, and the reward or penalty (negative reward) the environment gives to the agent, in return of the interaction.\
**States** are the information our agent gets from the environment. This must be a complete description of the state world, otherwise it is known as an **Observation** (partial description of the state)\
**Policy** a strategy which applies to the agent to decide the next action based on the current state/ observation\
**Reward** Immediate reward given when the agent performs a specific task\
**Value (V)** It is the expected long-term return with discount, as compared to the short-term reward.\

Discounting is the process of reducing the reward at each timestep to ensure that the agent reaches the optimal value/ policy in the least amount of steps.

There are two types of tasks that an RL algorithm can solve. The **Episodic Tasks** have a start and end point. **Continuing tasks** require the agent learn how to choose the best actions and simultaneously interact with the environment. The agent will keep running until it is told to stop

### RL Algorithms 
There are three approaches to finding solutions
- Value-Based: The goal is to try to maximize a value function V(s) 
- Policy-Based: Come up with a policy such that the  action performed in every state helps you to gain maximum reward in the future.
- Model-Based: Create a virtual model for each environment. The agent learns to perform in that specific environment. 

### Exploration vs Exploitation
Greedy and $\epsilon$ greedy

### Challenges of RL


## What is dynamic programming?

Dynamic Programming is a collection of algorithms created to explore environments that are finite **Markov Decision Process**. They are used typically to solve planning problems. Policy evaluation is the task of determining the state-value function $v_\pi$ for a given policy $\pi$. There are two types of Dynamic Programming algorithms
<ol>
  <li>Policy Iteration</li>
  <li>Value Iteration</li>

</ol> 
Both of these approaches are **Model Based** algorithms agent trying to understand its environment and creating a model for it based on its interactions with this environment. Let us begin implementing the Policy iteration algorithm in the pre-made OpenAI gym environment, FrozenLake-v1. We start by importing the necessary packages and rendering the environment. 

In [1]:
import numpy as np
import gym

In [10]:
env = gym.make('FrozenLake-v1', render_mode='human')
# enwrap it to have additional information from it
env = env.unwrapped
env.reset()
env.render()
# spaces dimension
nA = env.action_space.n
nS = env.observation_space.n

We start by choosing an arbitrary policy $\pi$ .We then iteratively evaluate and improve the policy based on the value function. 

In [11]:
# initializing value function and policy
V = np.zeros(nS)
policy = np.zeros(nS)

In [12]:
# some useful variable
policy_stable = False
it = 0

We can begin defining some functions necessary for the algorithm. The first function defines the value function (State-action pairs) of the various states

In [13]:
def eval_state_action(V, s, a, gamma=0.5):
    return np.sum([p * (rew + gamma*V[next_s]) for p, next_s, rew, _ in env.P[s][a]])

This next function allows us to evaluate our policy. Whether the policy is optimal is decided by this function

In [14]:
def policy_evaluation(V, policy, eps=0.0001):
    '''
    Policy evaluation. Update the value function until it reach a steady state
    '''
    while True:
        delta = 0
        # loop over all states
        for s in range(nS):
            old_v = V[s]
            # update V[s] using the Bellman equation
            V[s] = eval_state_action(V, s, policy[s])
            delta = max(delta, np.abs(old_v - V[s]))

        if delta < eps:
            break


This is the function we use to update the policy. 

In [15]:
def policy_improvement(V, policy):
    '''
    Policy improvement. Update the policy based on the value function
    '''
    policy_stable = True
    for s in range(nS):
        old_a = policy[s]
        # update the policy with the action that bring to the highest state value
        policy[s] = np.argmax([eval_state_action(V, s, a) for a in range(nA)])
        if old_a != policy[s]: 
            policy_stable = False

    return policy_stable

    while not policy_stable:
        policy_evaluation(V, policy)
        policy_stable = policy_improvement(V, policy)
        it += 1

Running the episode is equivalent to testing the agent. We want to know how well the agent will perform in the environment if it obeys the optimal policy. 

In [8]:
def run_episodes(env, policy, num_games=100):
    '''
    Run some games to test a policy
    '''
    tot_rew = 0
    state = env.reset()
    #print(type(state[0]))
    #print(type(policy))

    for _ in range(num_games):
        done = False
        while not done:
            # select the action accordingly to the policy
#             break
            next_state, reward, done,_,_ = env.step(int(policy[state[0]]))
            state=np.asarray(state)    
            state[0] = next_state
            print(state)
            tot_rew += reward 
            if done:
                state = env.reset()
                state=np.asarray(state)

            print('Won %i of %i games!'%(tot_rew, num_games))

In [ ]:
print('Converged after %i policy iterations'%(it))
run_episodes(env, policy)
print(V.reshape((4,4)))
print(policy.reshape((4,4)))

In [17]:
env.close()